modelo basado en estrategias
la idea es que ekl modelo detecte cuando es mejor usar cada estrategia


In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,root_mean_squared_error
import time
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import yfinance as yf

import pandas_ta as ta
import matplotlib.pyplot as plt


In [253]:
# Función para descargar datos OHLCV desde Binance
def get_binance_ohlcv(symbol, interval="1d", days=4000):
    """
    Descarga velas OHLCV desde Binance.
    symbol: par de trading (ej: BTCUSDT)
    interval: intervalo de vela (ej: 1h, 1d, 15m)
    days: días de datos hacia atrás
    """
    base_url = "https://api.binance.com/api/v3/klines"
    
    # Binance limita 1000 velas por request → dividir en chunks
    limit = 1000
    ms_interval = 60 * 60 * 1000  # 1h en milisegundos
    if interval == "1d":
        ms_interval = 24 * 60 * 60 * 1000
    
    end_time = int(time.time() * 1000)  # ahora en ms
    start_time = end_time - days * 24 * 60 * 60 * 1000
    
    all_data = []
    
    while start_time < end_time:
        params = {
            "symbol": symbol,
            "interval": interval,
            "startTime": start_time,
            "limit": limit
        }
        resp = requests.get(base_url, params=params)
        data = resp.json()
        
        if not data:
            break
        
        all_data.extend(data)
        
        # avanzar el start_time al último timestamp + intervalo
        last_open_time = data[-1][0]
        start_time = last_open_time + ms_interval
        
        time.sleep(0.2)  # para no sobrecargar la API
    
    # convertir a DataFrame
    df = pd.DataFrame(all_data, columns=[
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base", "taker_buy_quote", "ignore"
    ])
    
    # limpiar tipos de datos
    df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
    df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")
    for col in ["open", "high", "low", "close", "volume"]:
        df[col] = df[col].astype(float)
    
    return df[["open_time", "open", "high", "low", "close", "volume","number_of_trades"]]


In [669]:
#descarga de datos
df = get_binance_ohlcv("SOLUSDT", interval="1d", days=4000)

In [670]:
def crear_features_modelo(data, short_window=50, long_window=200, rsi_period=14, bollinger_window=20):
    """
    Calcula un conjunto de características numéricas (features) para un modelo de ML
    basado en estrategias de análisis técnico.
    
    Args:
        data (pd.DataFrame): DataFrame con datos OHLCV.
        
    Returns:
        pd.DataFrame: DataFrame con las nuevas características calculadas.
    """
    df = data.copy()
    
    # --- Feature 1: Retornos y volumen ---
    df['retorno_diario'] = df["close"].pct_change()
    df['cambio_volumen'] = df['volume'].pct_change()
    
    # --- Features de la Estrategia de Cruce de Medias Móviles ---
    sma_corta = df["close"].rolling(window=short_window).mean()
    sma_larga = df["close"].rolling(window=long_window).mean()
    
    df['distancia_smas'] = sma_corta - sma_larga
    # Evitamos dividir por cero si la SMA larga es 0 en algún punto inicial
    df['ratio_smas'] = sma_corta / sma_larga.replace(0, np.nan)
    
    # --- Features de la Estrategia de Bollinger + RSI ---
    # Calcular RSI
    df.ta.rsi(length=rsi_period, append=True)
    df.rename(columns={f'RSI_{rsi_period}': 'valor_rsi'}, inplace=True)
    
    # Calcular Bandas de Bollinger
    sma_bb = df["close"].rolling(window=bollinger_window).mean()
    std_bb = df["close"].rolling(window=bollinger_window).std()
    banda_sup = sma_bb + (std_bb * 2)
    banda_inf = sma_bb - (std_bb * 2)
    
    # Feature: Posición normalizada del precio dentro de las bandas
    # Un valor > 0 está por encima de la media, < 0 por debajo.
    df['posicion_en_bandas'] = (df["close"] - sma_bb) / std_bb.replace(0, np.nan)
    
    # Feature: Ancho de las bandas (mide volatilidad)
    df['ancho_bandas'] = (banda_sup - banda_inf) / sma_bb.replace(0, np.nan)
    

    
    # Eliminar filas con NaN generadas por los cálculos de ventanas móviles
    df.dropna(inplace=True)
    
    return df
df=crear_features_modelo(df)

In [671]:
#porcentaje de cambio de 1,7,14,30 dias
df['pct_change_1d'] = df['close'].pct_change(periods=1) * 100
df['pct_change_7d'] = df['close'].pct_change(periods=7) * 100
df['pct_change_14d'] = df['close'].pct_change(periods=14) * 100
df['pct_change_30d'] = df['close'].pct_change(periods=30) * 100


In [672]:
#calcular RSI
def compute_rsi(data, window=14):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi 
df['RSI'] = compute_rsi(df)

In [673]:
#bandas de bollinger
def compute_bollinger_bands(data, window=20, num_std=2):
    rolling_mean = data['close'].rolling(window=window).mean()
    rolling_std = data['close'].rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return rolling_mean, upper_band, lower_band
df['Banda_Media'], df['Banda_Superior'], df['Banda_Inferior'] = compute_bollinger_bands(df)

In [674]:
#medias moviles de 9,26,50,200
df['SMA_9'] = df['close'].rolling(window=9).mean()
df['SMA_26'] = df['close'].rolling(window=26).mean()
df['SMA_50'] = df['close'].rolling(window=50).mean()
df['SMA_200'] = df['close'].rolling(window=200).mean()


In [675]:
df.dropna(inplace=True)


In [676]:
#estrategia de bandas y RSI


def estrategia_bb_rsi(data, window=20, num_std=2, rsi_period=14, rsi_overbought=70, rsi_oversold=30):
    """
    Replica la estrategia de trading de reversión a la media con Bandas de Bollinger y RSI.
    
    Args:
        data (pd.DataFrame): DataFrame con datos de precios (debe tener 'close').
        window (int): Período para las Bandas de Bollinger.
        num_std (int): Número de desviaciones estándar para las bandas.
        rsi_period (int): Período para el RSI.
        rsi_overbought (int): Nivel de sobrecompra del RSI.
        rsi_oversold (int): Nivel de sobreventa del RSI.
        
    Returns:
        pd.DataFrame: DataFrame original con indicadores y la columna 'Señal'.
    """
    df = data.copy()
    
    # 1. Calcular Bandas de Bollinger
    df['SMA'] = df['close'].rolling(window=window).mean()
    df['STD'] = df['close'].rolling(window=window).std()
    df['Banda_Superior'] = df['SMA'] + (df['STD'] * num_std)
    df['Banda_Inferior'] = df['SMA'] - (df['STD'] * num_std)
    
    # 2. Calcular RSI usando pandas_ta
    # La librería añade automáticamente una columna 'RSI_14' al DataFrame
    df.ta.rsi(length=rsi_period, append=True)
    
    # Renombrar columna de RSI para claridad
    rsi_col_name = f'RSI_{rsi_period}' # El nombre por defecto es 'RSI_14'
    
    # 3. Definir las condiciones de la estrategia
    # Condición de compra: Precio toca o baja de la banda inferior Y RSI está en sobreventa.
    condicion_compra = (df['close'] <= df['Banda_Inferior']) & (df[rsi_col_name] < rsi_oversold)
    
    # Condición de venta: Precio toca o supera la banda superior Y RSI está en sobrecompra.
    condicion_venta = (df['close'] >= df['Banda_Superior']) & (df[rsi_col_name] > rsi_overbought)
    
    # 4. Crear la columna de Señal
    df['Señal_ByR'] = np.nan
    df.loc[condicion_compra, 'Señal_ByR'] = 1  # Señal de ALZA (Compra)
    df.loc[condicion_venta, 'Señal_ByR'] = 0   # Señal de BAJA (Venta)
    
    return df
df = estrategia_bb_rsi(df)

In [677]:
#estrategia de cruce de medias móviles (Golden/Death Cross)

def estrategia_cruce_medias(data, short_window=50, long_window=200):
    """
    Replica la estrategia de cruce de medias móviles (Golden/Death Cross).
    
    Args:
        data (pd.DataFrame): DataFrame con los datos de precios (debe tener 'close').
        short_window (int): Periodo para la media móvil corta.
        long_window (int): Periodo para la media móvil larga.
        
    Returns:
        pd.DataFrame: El DataFrame original con las medias móviles y la columna 'Señal'.
    """
    # Se crea una copia para no modificar el df original
    df = data.copy()
    
    # 1. Calcular las Medias Móviles Simples (SMA)
    df['SMA_Corta'] = df['close'].rolling(window=short_window, min_periods=1).mean()
    df['SMA_Larga'] = df['close'].rolling(window=long_window, min_periods=1).mean()
    
    # 2. Definir las condiciones de cruce
    # Se compara la posición de hoy con la de ayer (.shift(1)) para encontrar el punto exacto del cruce.
    
    # Golden Cross (Cruce Alcista): La SMA corta cruza por encima de la SMA larga.
    condicion_alza = (df['SMA_Corta'] > df['SMA_Larga']) & (df['SMA_Corta'].shift(1) <= df['SMA_Larga'].shift(1))
    
    # Death Cross (Cruce Bajista): La SMA corta cruza por debajo de la SMA larga.
    condicion_baja = (df['SMA_Corta'] < df['SMA_Larga']) & (df['SMA_Corta'].shift(1) >= df['SMA_Larga'].shift(1))
    
    # 3. Crear la columna de Señal
    # Se inicializa con NaN (sin señal)
    df['Señal'] = np.nan 
    # Se asigna 1 en los puntos de cruce alcista
    df.loc[condicion_alza, 'Señal'] = 1
    # Se asigna 0 en los puntos de cruce bajista
    df.loc[condicion_baja, 'Señal'] = 0
    
    return df
df = estrategia_cruce_medias(df, short_window=50, long_window=200)

In [678]:
df.reset_index(drop=True, inplace=True)

In [679]:
len(df)

1484

In [680]:
df

,open_time,open,high,low,close,volume,number_of_trades,retorno_diario,cambio_volumen,distancia_smas,...,SMA_26,SMA_50,SMA_200,SMA,STD,RSI_14,Señal_ByR,SMA_Corta,SMA_Larga,Señal
0,2021-09-13,174.31,175.00,150.30,169.53,8423839.460,1867633,-0.027311,1.300834,39.762823,...,122.048308,83.04514,43.282317,NaN,NaN,NaN,NaN,169.5300,169.53000,NaN
1,2021-09-14,169.54,171.83,142.60,158.88,7981396.520,1750566,-0.062821,-0.052523,41.650325,...,125.360462,85.65906,44.008736,NaN,NaN,0.000000,NaN,164.2050,164.20500,NaN
2,2021-09-15,158.72,166.50,153.69,158.79,6107319.600,1219649,-0.000566,-0.234806,43.536072,...,128.443038,88.27280,44.736728,NaN,NaN,0.000000,NaN,162.4000,162.40000,NaN
3,2021-09-16,158.77,163.50,147.35,152.29,4739595.730,973309,-0.040935,-0.223948,45.328366,...,131.461846,90.76110,45.432734,NaN,NaN,0.000000,NaN,159.8725,159.87250,NaN
4,2021-09-17,152.30,153.55,134.00,147.58,7629925.550,1405718,-0.030928,0.609826,46.992293,...,134.338962,93.08800,46.095707,NaN,NaN,0.000000,NaN,157.4140,157.41400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,2025-10-01,208.68,222.77,206.31,222.04,4050260.921,2119007,0.064021,0.240592,42.870100,...,222.833846,210.48860,167.618500,225.1525,16.738134,53.332212,NaN,210.4886,167.61850,NaN
1480,2025-10-02,222.04,234.99,219.52,234.74,4590696.388,2550516,0.057197,0.133432,42.991650,...,224.163846,211.15320,168.161550,224.7805,16.419427,59.188970,NaN,211.1532,168.16155,NaN
1481,2025-10-03,234.74,237.00,227.13,232.84,4216611.614,2632758,-0.008094,-0.081488,43.274900,...,225.182692,211.96060,168.685700,224.3035,16.014823,58.015906,NaN,211.9606,168.68570,NaN
1482,2025-10-04,232.84,234.05,224.07,227.86,1841410.605,1292594,-0.021388,-0.563296,43.605050,...,225.711538,212.80340,169.198350,223.7085,15.626700,54.942300,NaN,212.8034,169.19835,NaN


In [681]:
df.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume',
       'number_of_trades', 'retorno_diario', 'cambio_volumen',
       'distancia_smas', 'ratio_smas', 'valor_rsi', 'posicion_en_bandas',
       'ancho_bandas', 'pct_change_1d', 'pct_change_7d', 'pct_change_14d',
       'pct_change_30d', 'RSI', 'Banda_Media', 'Banda_Superior',
       'Banda_Inferior', 'SMA_9', 'SMA_26', 'SMA_50', 'SMA_200', 'SMA', 'STD',
       'RSI_14', 'Señal_ByR', 'SMA_Corta', 'SMA_Larga', 'Señal'],
      dtype='object')

In [682]:
horizonte = 1
df['Target'] = (df['close'].shift(-horizonte) > df['close']).astype(int)

In [683]:
#df=pd.read_csv("data/raw/df_criptos/df_ETHUSDT.csv")

In [684]:
df

,open_time,open,high,low,close,volume,number_of_trades,retorno_diario,cambio_volumen,distancia_smas,...,SMA_50,SMA_200,SMA,STD,RSI_14,Señal_ByR,SMA_Corta,SMA_Larga,Señal,Target
0,2021-09-13,174.31,175.00,150.30,169.53,8423839.460,1867633,-0.027311,1.300834,39.762823,...,83.04514,43.282317,NaN,NaN,NaN,NaN,169.5300,169.53000,NaN,0
1,2021-09-14,169.54,171.83,142.60,158.88,7981396.520,1750566,-0.062821,-0.052523,41.650325,...,85.65906,44.008736,NaN,NaN,0.000000,NaN,164.2050,164.20500,NaN,0
2,2021-09-15,158.72,166.50,153.69,158.79,6107319.600,1219649,-0.000566,-0.234806,43.536072,...,88.27280,44.736728,NaN,NaN,0.000000,NaN,162.4000,162.40000,NaN,0
3,2021-09-16,158.77,163.50,147.35,152.29,4739595.730,973309,-0.040935,-0.223948,45.328366,...,90.76110,45.432734,NaN,NaN,0.000000,NaN,159.8725,159.87250,NaN,0
4,2021-09-17,152.30,153.55,134.00,147.58,7629925.550,1405718,-0.030928,0.609826,46.992293,...,93.08800,46.095707,NaN,NaN,0.000000,NaN,157.4140,157.41400,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,2025-10-01,208.68,222.77,206.31,222.04,4050260.921,2119007,0.064021,0.240592,42.870100,...,210.48860,167.618500,225.1525,16.738134,53.332212,NaN,210.4886,167.61850,NaN,1
1480,2025-10-02,222.04,234.99,219.52,234.74,4590696.388,2550516,0.057197,0.133432,42.991650,...,211.15320,168.161550,224.7805,16.419427,59.188970,NaN,211.1532,168.16155,NaN,0
1481,2025-10-03,234.74,237.00,227.13,232.84,4216611.614,2632758,-0.008094,-0.081488,43.274900,...,211.96060,168.685700,224.3035,16.014823,58.015906,NaN,211.9606,168.68570,NaN,0
1482,2025-10-04,232.84,234.05,224.07,227.86,1841410.605,1292594,-0.021388,-0.563296,43.605050,...,212.80340,169.198350,223.7085,15.626700,54.942300,NaN,212.8034,169.19835,NaN,0


In [685]:
len(df)

1484

In [688]:
#train y test 
relativ_error=[]
corr_direccion=0
aciertos_direccion_list=[]
error_aciertos_list=[]
y_pred_guardadas=[]
y_real_guardadas=[]
restricciones=['open_time', 'open',  'close', 'volume',
       'retorno_diario', 
       'distancia_smas', 'ratio_smas', 'valor_rsi', 
       'ancho_bandas', 'pct_change_1d', 'pct_change_7d', 'pct_change_14d',
       'pct_change_30d', 'SMA_9', 'SMA_26', 'SMA_50', 'SMA_200', 'Target',
       'SMA', 'STD', 
       'SMA_Corta', 'SMA_Larga']#'number_of_trades'

medir='Target'
for i in range(100):
    print(i)
    #vamos a separar los datos de entreno y los outsider
    entreno=df[0+i:1383+i]
    outsider=df[1383+i+1:1383+i+2]
    #print(outsider)
    outsider.reset_index(drop=True,inplace=True)
    entreno.reset_index(drop=True,inplace=True)
    #ahora ponemos el y el x
    X_entreno=entreno.drop(restricciones,axis=1)
    y=entreno[medir]
    X_train, X_test, y_train, y_test = train_test_split(X_entreno, y, test_size=0.2, random_state=42)
    #model=RandomForestRegressor( n_estimators=100, random_state=42)
    model=RandomForestClassifier( random_state=42,n_estimators= 30, max_depth= 1, min_samples_split= 2, min_samples_leaf= 3)



    model.fit(X_train,y_train)
    X_outsider=outsider.drop(restricciones,axis=1)
    y_outsider=outsider[medir]#es el real
    predicciones_outsider=model.predict(X_outsider)
    #print(y_outsider[0])
    y_pred_guardadas.append(predicciones_outsider[0])
    y_real_guardadas.append(y_outsider[0])
    #error_relativo_porcentual=abs(y_outsider[0]-predicciones_outsider)/y_outsider[0]
    #relativ_error.append(error_relativo_porcentual)
    #if y_outsider[0]*predicciones_outsider>0:
        #aciertos_direccion_list.append(predicciones_outsider)
        #error_relativo_aciertos=abs(y_outsider[0]-predicciones_outsider)/y_outsider[0]
        #error_aciertos_list.append(error_relativo_aciertos)

#error_total=abs(np.array(relativ_error).sum())/len(relativ_error)
#aciertos_direccion=len(aciertos_direccion_list)
#error_aciertos=abs(np.array(error_aciertos_list).sum())/len(error_aciertos_list)

    if predicciones_outsider[0]==y_outsider[0]:
        corr_direccion+=1
print(corr_direccion/100)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.46


In [ ]:
#df.to_csv("/home/rodrigo/Escritorio/Repositorios/Proyecto_cripto_RH/data/raw/df_criptos/df_BTCUSDT.csv", index=False)

restricciones del 64%
'open_time', 'open', 'high', 'low', 'volume',
       'number_of_trades',  
       'pct_change_14d','Banda_Media', 
        'SMA_9', 'SMA_26', 'SMA_50', 'SMA_200', 'SMA', 'STD',
       'RSI_14',"direccion_pred"